# Notebook 12: Identifying Phases in the 2D Ising Model with TensorFlow


In [1]:
from __future__ import absolute_import, division, print_function
import numpy as np
import sys, os, argparse
import tensorflow as tf
from tensorflow.python.framework import dtypes

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import pickle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import torch
from torch.utils.tensorboard import SummaryWriter

seed=12
np.random.seed(seed)
tf.random.set_seed(seed)
writer = SummaryWriter()

### Step 1: Load and Process the Data

In [2]:
import pickle, os
from urllib.request import urlopen 

def load_data():
    url_main = 'https://physics.bu.edu/~pankajm/ML-Review-Datasets/isingMC/';
    data_file_name = "Ising2DFM_reSample_L40_T=All.pkl" 
    label_file_name = "Ising2DFM_reSample_L40_T=All_labels.pkl"

    data = pickle.load(urlopen(url_main + data_file_name)) # pickle reads the file and returns the Python object (1D array, compressed bits)
    data = np.unpackbits(data).reshape(-1, 1600) # Decompress array and reshape for convenience
    data=data.astype('int')
    data[np.where(data==0)]=-1 # map 0 state to -1 (Ising variable can take values +/-1)

    labels = pickle.load(urlopen(url_main + label_file_name)) # pickle reads the file and returns the Python object (here just a 1D array with the binary labels)
    
    print("Finished loading data")
    return data, labels

def prepare_data(data, labels, dtype=dtypes.float32, test_size=0.2, validation_size=5000):
    
    L=40 # linear system size

    X_ordered=data[:70000,:].reshape(-1,40,40,1)
    Y_ordered=labels[:70000]

    X_critical=data[70000:100000,:].reshape(-1,40,40,1)
    Y_critical=labels[70000:100000]

    X_disordered=data[100000:,:].reshape(-1,40,40,1)
    Y_disordered=labels[100000:]

    X=np.concatenate((X_ordered,X_disordered)) #np.concatenate((X_ordered,X_critical,X_disordered))
    Y=np.concatenate((Y_ordered,Y_disordered)) #np.concatenate((Y_ordered,Y_critical,Y_disordered))

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, train_size=1.0-test_size)

    Y_train=to_categorical(Y_train)
    Y_test=to_categorical(Y_test)
    Y_critical=to_categorical(Y_critical)

    if not 0 <= validation_size <= len(X_train):
        raise ValueError('Validation size should be between 0 and {}. Received: {}.'.format(len(X_train), validation_size))

    X_validation = X_train[:validation_size]
    Y_validation = Y_train[:validation_size]
    X_train = X_train[validation_size:]
    Y_train = Y_train[validation_size:]

    dataset = {
        'train': (X_train, Y_train),
        'test': (X_test, Y_test),
        'critical': (X_critical, Y_critical),
        'validation': (X_validation, Y_validation)
    }
    return dataset

def prepare_Ising_DNN():
    data, labels = load_data()
    return prepare_data(data, labels, test_size=0.2, validation_size=5000)

IsingCNN = prepare_Ising_DNN()

Finished loading data


## Creating Convolutional Neural Nets with TensorFlow

In [3]:
def create_CNN():
    model = Sequential()
    model.add(Conv2D(10, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=(40,40,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(20, (5, 5), activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer='Adam',
                  metrics=['accuracy'])
    return model

In [4]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

# training parameters
batch_size = 128
epochs = 2

# create the deep conv net
model_CNN=create_CNN()

# train CNN
model_CNN.fit(*IsingCNN["train"],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=IsingCNN["validation"],
          callbacks=[tensorboard_callback])

# evaliate model
score = model_CNN.evaluate(*IsingCNN["test"], verbose=1)
score_critical = model_CNN.evaluate(*IsingCNN["critical"], verbose=1)

print('\nTest loss:', score[0])
print('Test accuracy:', score[1])
print('Critical data loss:', score_critical[0])
print('Critical data accuracy:', score_critical[1])

Epoch 1/2
774/774 [==============================] - 55s 70ms/step - loss: 0.0048 - accuracy: 0.9981 - val_loss: 2.4930e-06 - val_accuracy: 1.0000
Epoch 2/2
938/938 [==============================] - 7s 8ms/step - loss: 0.2074 - accuracy: 0.9451

Test loss: 2.8447396971387207e-07
Test accuracy: 1.0
Critical data loss: 0.20744609832763672
Critical data accuracy: 0.9451333284378052


In [9]:
%load_ext tensorboard
%tensorboard --logdir="./logs" --host localhost

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
